# Conjunto de Dados 3: *VinBigData Dataset*
***
* Disponível em: <https://www.kaggle.com/awsaf49/vinbigdata-original-image-dataset>. Acesso em 24 fev. 2021.

### Importação dos pacotes

In [31]:
from keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import train_test_split
from sklearn.utils import class_weight
import pandas as pd
import numpy as np
import os

### Pré-processamento dos dados

In [4]:
dataframe = pd.read_csv('../0-datasets/vinbigdata/arquivos-descompactados/vinbigdata/train.csv')
path = '../0-datasets/vinbigdata/arquivos-descompactados/vinbigdata/train/'
dataframe['image_path'] = path + dataframe.image_id + '.jpg'

print('total de imagens disponíveis:', str(len(set(dataframe['image_path']))))

total de imagens disponíveis: 15000


In [6]:
dataframe['class_name'].value_counts()

No finding            31818
Aortic enlargement     7162
Cardiomegaly           5427
Pleural thickening     4842
Pulmonary fibrosis     4655
Nodule/Mass            2580
Lung Opacity           2483
Pleural effusion       2476
Other lesion           2203
Infiltration           1247
ILD                    1000
Calcification           960
Consolidation           556
Atelectasis             279
Pneumothorax            226
Name: class_name, dtype: int64

In [8]:
dataframe = dataframe[dataframe.class_name != 'Aortic enlargement']
dataframe = dataframe[dataframe.class_name != 'Cardiomegaly']
dataframe = dataframe[dataframe.class_name != 'Other lesion']
dataframe = dataframe[dataframe.class_name != 'Consolidation']

normal_cases = dataframe[(dataframe.class_id == 14) & (dataframe.class_name == 'No finding')]
abnormal_cases = dataframe[(dataframe.class_id != 14) & (dataframe.class_name != 'No finding')]
lenght_images = len(set(normal_cases['image_path'])) + len(set(abnormal_cases['image_path']))
print('total de dados após a filtração: {}'.format(lenght_images))

total de dados após a filtração: 13948


In [10]:
normal_data = normal_cases[['image_path', 'class_name']].drop_duplicates(subset = 'image_path')
abnormal_data = abnormal_cases[['image_path', 'class_name']].drop_duplicates(subset = 'image_path')
normal_data['target'] = 'normal'
abnormal_data['target'] = 'abnormal'

print('quantidade de dados rotulados como normais:', len(normal_data))
print('quantidade de dados rotulados como anormais:', len(abnormal_data))

quantidade de dados rotulados como normais: 10606
quantidade de dados rotulados como anormais: 3342


In [12]:
normal, _ = train_test_split(normal_data, test_size = 0.69, random_state = 42)

print('quantidade de dados rotulados como normais:', len(normal))
print('quantidade de dados rotulados como anormais:', len(abnormal_data))

quantidade de dados rotulados como normais: 3287
quantidade de dados rotulados como anormais: 3342


In [18]:
full_data = pd.concat([normal, abnormal_data])
train_df, test_df = train_test_split(full_data, stratify = full_data['target'], test_size = 0.2, random_state = 42)
train_df, validation_df = train_test_split(train_df, stratify = train_df['target'], test_size = 0.2, random_state = 42)

print('quantidade de imagens de treinamento:', len(train_df['image_path']))
print('quantidade de rótulos de treinamento:', len(train_df['target']))
print('quantidade de imagens de teste:', len(test_df['image_path']))
print('quantidade de rótulos de teste:', len(test_df['target']))
print('quantidade de imagens de validação:', len(validation_df['image_path']))
print('quantidade de rótulos de validação:', len(validation_df['target']))

quantidade de imagens de treinamento: 4242
quantidade de rótulos de treinamento: 4242
quantidade de imagens de teste: 1326
quantidade de rótulos de teste: 1326
quantidade de imagens de validação: 1061
quantidade de rótulos de validação: 1061


In [24]:
class_weights = class_weight.compute_class_weight('balanced', np.unique(train_df['class_name']), train_df['class_name'])
class_weight = {0: class_weights[0], 1: class_weights[1], 2: class_weights[2], 
                3: class_weights[3], 4: class_weights[4], 5: class_weights[5],
                6: class_weights[6], 7: class_weights[7], 8: class_weights[8],
                9: class_weights[9], 10: class_weights[10]}
                
train_df.to_csv('train_df.csv')
validation_df.to_csv('validation_df.csv')
test_df.to_csv('test_df.csv')

### Gerador de dados para a rede pelo Tensorflow

In [21]:
image_generator = ImageDataGenerator(samplewise_center = True, samplewise_std_normalization = True,
                                     rotation_range = 10, zoom_range = 0.2)

train_generator = image_generator.flow_from_dataframe(dataframe = train_df, directory = '', x_col = 'image_path',
                                                      y_col = 'target', batch_size = 32, seed = 42,
                                                      shuffle = True, class_mode = 'categorical',
                                                      target_size = (256, 256))
 
valid_generator = image_generator.flow_from_dataframe(dataframe = validation_df, directory = '.', x_col = 'image_path', 
                                                      y_col = 'target', batch_size = 32, seed = 42,
                                                      shuffle = True, class_mode = 'categorical',
                                                      target_size = (256, 256))

test_datagen = ImageDataGenerator(samplewise_center = True, samplewise_std_normalization = True)

test_generator = test_datagen.flow_from_dataframe(dataframe = test_df, directory = '.', x_col = 'image_path',
                                                  y_col = 'target', batch_size = 32, seed = 42,
                                                  shuffle = True, class_mode = 'categorical',
                                                  target_size = (256, 256))

Found 4242 validated image filenames belonging to 2 classes.
Found 1061 validated image filenames belonging to 2 classes.
Found 1326 validated image filenames belonging to 2 classes.


In [23]:
train_generator.class_indices

{'abnormal': 0, 'normal': 1}